### Pipeline that Generates AP and GP and Calculates the Sum and 'r'th term

In [ ]:
# The kfp-python package provides the Interface to Generate and Run Pipelines

import kfp
import typing

#### Read Inputs for Arithmatic Progression

In [ ]:
from typing import NamedTuple
from kfp.components import *

def read_inputs_ap(first_term: float, difference: float, number_of_terms: int, find_rth_term: int) -> NamedTuple('Outputs',
                                                                                             [('a', float), 
                                                                                         ('d', float), 
                                                                                              ('n', int), ('r', int)]):
    a = first_term
    d = difference
    n = number_of_terms
    r = find_rth_term
    return (a, d, n, r)


In [ ]:
kfp_read_inputs_ap = kfp.components.func_to_container_op(func = read_inputs_ap, 
                                                          output_component_file = './read_inp_ap.yaml')


#### Compute the AP Series Terms

In [ ]:
def ap_series(a : float, d : float, n : int)-> NamedTuple('Outputs', [('a', float), ('d', float), ('n', int),
                                                         ('ser', list)]):
    l = []
    for x in range(n):
        l.append(a + x*d)
    
    print("First Term : {}\nCommon Difference : {}\nLength of Series : {}\nLast Term : {}".format(a,d,n,l[-1]))
    print("Series :")
    for x in l:
        print("{}  ".format(x), end =" ")
    
    return (a, d, n, l)

In [ ]:
kfp_ap_series = kfp.components.func_to_container_op(func = ap_series, 
                                                          output_component_file = './ap_series.yaml')


#### Calculate the Sum of AP terms

In [ ]:
def ap_sum(a : float, d : float, n : int, ser : list)-> float:
    
    sum_of_ap = n*(2*a + (n-1)*d)/2
    print("Sum of the AP is: {}".format(sum_of_ap))
    return sum_of_ap

In [ ]:
kfp_ap_sum = kfp.components.func_to_container_op(func = ap_sum, 
                                                          output_component_file = './ap_sum.yaml')


#### Compute the 'r'th term

In [ ]:
def ap_rth_term(a : float, d : float, r : int)-> float:
    r_term = a + (r-1)*d
    
    if r>3:
        print("{}th term of AP is : {}".format(r, r_term))
    elif r==2:
        print("{}nd term of AP is : {}".format(r, r_term))
    elif r==1:
        print("{}st term of AP is : {}".format(r, r_term))
    
    return r_term

In [ ]:
kfp_ap_rth_term = kfp.components.func_to_container_op(func = ap_rth_term, 
                                                          output_component_file = './ap_rth_term.yaml')


#### Read Inputs for Geometric Progression

In [ ]:
def read_inputs_gp(first_term: float, common_ratio: float, number_of_terms: int, find_rth_term: int) -> NamedTuple('Outputs',
                                                                                             [('a', float), 
                                                                                         ('d', float), 
                                                                                              ('n', int), ('r', int)]):
    
    a = first_term
    d = common_ratio
    n = number_of_terms
    r = find_rth_term
    return (a, d, n, r)


In [ ]:
kfp_read_inputs_gp = kfp.components.func_to_container_op(func = read_inputs_gp, 
                                                          output_component_file = './read_inp_gp.yaml')


#### Compute the GP Series Terms

In [ ]:
def gp_series(a : float, d : float, n : int)-> NamedTuple('Outputs', [('a', float), ('d', float), ('n', int),
                                                         ('ser', list)]):
    l = []
    for x in range(n):
        l.append(a*pow(d,x))
    
    print("First Term : {}\nCommon Ratio : {}\nLength of Series : {}\nLast Term : {}".format(a,d,n,l[-1]))
    print("Series :")
    
    for x in l:
        print("{}  ".format(x), end =" ")
    
    return (a, d, n, l)

In [ ]:
kfp_gp_series = kfp.components.func_to_container_op(func = gp_series, 
                                                          output_component_file = './gp_series.yaml')


#### Calculate the Sum of GP terms

In [ ]:
#### Calculate the Sum of AP terms
def gp_sum(a : float, d : float, n : int, ser : list)-> float:
    
    sum_of_gp = a*(1-pow(d,n))/(1-d)
    print("Sum of the GP is : {}".format(sum_of_gp))
    return sum_of_gp

In [ ]:
kfp_gp_sum = kfp.components.func_to_container_op(func = gp_sum, 
                                                          output_component_file = './gp_sum.yaml')


#### Compute the 'r'th term

In [ ]:
def gp_rth_term(a : float, d : float, r : int)-> float:
    r_term = a*pow(d,(r-1))
    
    if r>3:
        print("{}th term of GP is : {}".format(r, r_term))
    elif r==2:
        print("{}nd term of GP is : {}".format(r, r_term))
    elif r==1:
        print("{}st term of GP is : {}".format(r, r_term))

    return r_term

In [ ]:
kfp_gp_rth_term = kfp.components.func_to_container_op(func = gp_rth_term, 
                                                          output_component_file = './gp_rth_term.yaml')


#### Defining the Pipeline Execution Sequence and Input-Output scheme

In [ ]:
import kfp.dsl as dsl

@dsl.pipeline(name='AP-GP Pipeline',description='Calculates Simple and Compund Interest')
def Apgp_func(first_term_ap: float, common_difference_ap: float, total_terms_ap : int, rth_term_ap : int, 
              first_term_gp: float, common_ratio_gp: float, total_terms_gp : int, rth_term_gp : int):
    
    #Passing pipeline parameter and a constant value as operation arguments
    kfp_read_inputs_ap_task = kfp_read_inputs_ap(first_term_ap, common_difference_ap, total_terms_ap, rth_term_ap) 
    kfp_ap_series_task = kfp_ap_series(kfp_read_inputs_ap_task.outputs['a'], kfp_read_inputs_ap_task.outputs['d'],
                                      kfp_read_inputs_ap_task.outputs['n'])
    kfp_ap_sum_task = kfp_ap_sum(kfp_read_inputs_ap_task.outputs['a'], kfp_read_inputs_ap_task.outputs['d'],
                                      kfp_read_inputs_ap_task.outputs['n'], kfp_ap_series_task.outputs['ser'])
    kfp_ap_rth_term_task = kfp_ap_rth_term(kfp_read_inputs_ap_task.outputs['a'], kfp_read_inputs_ap_task.outputs['d'],
                                      kfp_read_inputs_ap_task.outputs['r'])
    
    kfp_read_inputs_gp_task = kfp_read_inputs_gp(first_term_gp, common_ratio_gp, total_terms_gp, rth_term_gp) 
    kfp_gp_series_task = kfp_gp_series(kfp_read_inputs_gp_task.outputs['a'], kfp_read_inputs_gp_task.outputs['d'],
                                      kfp_read_inputs_gp_task.outputs['n'])
    kfp_gp_sum_task = kfp_gp_sum(kfp_read_inputs_gp_task.outputs['a'], kfp_read_inputs_gp_task.outputs['d'],
                                      kfp_read_inputs_gp_task.outputs['n'], kfp_gp_series_task.outputs['ser'])
    kfp_gp_rth_term_task = kfp_gp_rth_term(kfp_read_inputs_gp_task.outputs['a'], kfp_read_inputs_gp_task.outputs['d'],
                                      kfp_read_inputs_gp_task.outputs['r'])
    
#For an operation with a single return value, the output reference can be accessed using `task.output` or `task.outputs['output_name']` syntax
#For an operation with a multiple return values, the output references can be accessed using `task.outputs['output_name']` syntax

#### Compiling the Pipeline

In [ ]:
pipeline_func = Apgp_func
pipeline_filename = pipeline_func.__name__+'.pipeline.tar.gz'

import kfp.compiler as comp
comp.Compiler().compile(pipeline_func, pipeline_filename)